## Assignment 2
  
- 문제에 필요한 데이터는 KMRD-small 사용
- 사용자 수는 2,000명으로 제한하라.
- RMSE평가를 위해서는 데이터를 학습용과 test용으로 분리하라.
    - `train_test_split(ratings_df, test_size=0.2, random_state=1234)`

In [2]:
import pandas as pd
import numpy as np

## 문제 1번 : KMRD-small에 대해 pandas table을 구해라  
#### (1) 다음 그림과 같은 movietable을 구하라.

#### 데이터 불러오기

In [38]:
# 파일 경로 지정
movies_file_path = '/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/kmrd-small/movies.txt'
peoples_file_path = '/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/kmrd-small/peoples.txt'
countries_file_path = '/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/kmrd-small/countries.csv'
genres_file_path = '/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/kmrd-small/genres.csv'
castings_file_path = '/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/kmrd-small/castings.csv'
rates_file_path = '/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/kmrd-small/rates.csv'

In [13]:
m_cols = ['movie', 'title', 'title_eng', 'year', 'grade']
movies = pd.read_csv(movies_file_path, sep='\t', names=m_cols, skiprows=1)
movies.head()

,movie,title,title_eng,year,grade
0,10001,시네마 천국,"Cinema Paradiso , 1988",2013.0,전체 관람가
1,10002,빽 투 더 퓨쳐,"Back To The Future , 1985",2015.0,12세 관람가
2,10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989",2015.0,12세 관람가
3,10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990",1990.0,전체 관람가
4,10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977",1997.0,PG


In [14]:
g_cols = ['movie', 'genres']
genres = pd.read_csv(genres_file_path, sep=',', names=g_cols, skiprows=1)
genres.head()

,movie,genres
0,10001,드라마
1,10001,멜로/로맨스
2,10002,SF
3,10002,코미디
4,10003,SF


In [15]:
c_cols = ['movie', 'country']
counties = pd.read_csv(countries_file_path, sep=',', names=c_cols, skiprows=1)
counties.head()

,movie,country
0,10001,이탈리아
1,10001,프랑스
2,10002,미국
3,10003,미국
4,10004,미국


In [16]:
p_cols = ['people','korean','original']
peoples = pd.read_csv(peoples_file_path, sep='\t', names=p_cols, skiprows=1)
peoples.head()

,people,korean,original
0,5,아담 볼드윈,Adam Baldwin
1,8,애드리안 라인,Adrian Lyne
2,9,에이단 퀸,Aidan Quinn
3,13,구로사와 아키라,Akira Kurosawa
4,15,알 파치노,Al Pacino


In [17]:
ca_cols = ['movie', 'people', 'order', 'leading']
castings = pd.read_csv(castings_file_path, sep=',', names=ca_cols, skiprows=1)

# 크레딧 순서와 주연에 관련한 columns는 불필요함으로 삭제
castings = castings.drop('order', axis=1)
castings = castings.drop('leading', axis=1)
castings.head()

,movie,people
0,10001,4374
1,10001,178
2,10001,3241
3,10001,47952
4,10001,47953


#### 영화별 캐스팅된 배우를 확인하기 위한 DataFrame 병합

In [18]:
# castings와 peoples 데이터프레임 병합
# 'people' 열을 기준으로 내부 조인을 수행
merged_people_data = pd.merge(castings, peoples, on='people', how='inner')

# 결과 출력
merged_people_data.head()

,movie,people,korean,original
0,10001,4374,자끄 페렝,Jacques Perrin
1,10001,178,브리지트 포시,Brigitte Fossey
2,10001,3241,필립 느와레,Philippe Noiret
3,10001,47952,살바토레 카스치오,Salvatore Cascio
4,10001,47953,안토넬라 아틸리,Antonella Attili


#### 최종 movietable 병합

In [19]:
# 영화데이터랑 장르데이터 병합
group_genres = genres.groupby('movie')['genres'].agg(list).reset_index()
merged_movie_table = pd.merge(movies, group_genres, on='movie', how='left')
merged_movie_table.head()

,movie,title,title_eng,year,grade,genres
0,10001,시네마 천국,"Cinema Paradiso , 1988",2013.0,전체 관람가,"[드라마, 멜로/로맨스]"
1,10002,빽 투 더 퓨쳐,"Back To The Future , 1985",2015.0,12세 관람가,"[SF, 코미디]"
2,10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989",2015.0,12세 관람가,"[SF, 코미디]"
3,10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990",1990.0,전체 관람가,"[서부, SF, 판타지, 코미디]"
4,10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977",1997.0,PG,"[판타지, 모험, SF, 액션]"


In [20]:
# 영화데이터와 배우(castings + peoples)데이터 병합
group_peoples = merged_people_data.groupby('movie')['people'].agg(list).reset_index()
merged_movie_table = pd.merge(merged_movie_table, group_peoples, on='movie', how='left')
merged_movie_table.head()

,movie,title,title_eng,year,grade,genres,people
0,10001,시네마 천국,"Cinema Paradiso , 1988",2013.0,전체 관람가,"[드라마, 멜로/로맨스]","[4374, 178, 3241, 47952, 47953, 19538, 18991, ..."
1,10002,빽 투 더 퓨쳐,"Back To The Future , 1985",2015.0,12세 관람가,"[SF, 코미디]","[1076, 4603, 917, 8637, 5104, 9986, 7470, 9987]"
2,10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989",2015.0,12세 관람가,"[SF, 코미디]","[1076, 4603, 917, 5104, 391, 5106, 5105, 5107,..."
3,10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990",1990.0,전체 관람가,"[서부, SF, 판타지, 코미디]","[1076, 4603, 1031, 5104, 10001, 5984, 10002, 1..."
4,10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977",1997.0,PG,"[판타지, 모험, SF, 액션]","[1007, 535, 215, 1236, 35]"


In [21]:
# 영화데이터와 country데이터 병합
group_countries = counties.groupby('movie')['country'].agg(','.join).reset_index()
merged_movie_table = pd.merge(merged_movie_table, group_countries, on='movie', how='left')
merged_movie_table.head()

,movie,title,title_eng,year,grade,genres,people,country
0,10001,시네마 천국,"Cinema Paradiso , 1988",2013.0,전체 관람가,"[드라마, 멜로/로맨스]","[4374, 178, 3241, 47952, 47953, 19538, 18991, ...","이탈리아,프랑스"
1,10002,빽 투 더 퓨쳐,"Back To The Future , 1985",2015.0,12세 관람가,"[SF, 코미디]","[1076, 4603, 917, 8637, 5104, 9986, 7470, 9987]",미국
2,10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989",2015.0,12세 관람가,"[SF, 코미디]","[1076, 4603, 917, 5104, 391, 5106, 5105, 5107,...",미국
3,10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990",1990.0,전체 관람가,"[서부, SF, 판타지, 코미디]","[1076, 4603, 1031, 5104, 10001, 5984, 10002, 1...",미국
4,10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977",1997.0,PG,"[판타지, 모험, SF, 액션]","[1007, 535, 215, 1236, 35]",미국


#### movie를 인덱스로 설정

In [22]:
set_movie_table = merged_movie_table.set_index('movie')
set_movie_table

,title,title_eng,year,grade,genres,people,country
movie,,,,,,,
10001,시네마 천국,"Cinema Paradiso , 1988",2013.0,전체 관람가,"[드라마, 멜로/로맨스]","[4374, 178, 3241, 47952, 47953, 19538, 18991, ...","이탈리아,프랑스"
10002,빽 투 더 퓨쳐,"Back To The Future , 1985",2015.0,12세 관람가,"[SF, 코미디]","[1076, 4603, 917, 8637, 5104, 9986, 7470, 9987]",미국
10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989",2015.0,12세 관람가,"[SF, 코미디]","[1076, 4603, 917, 5104, 391, 5106, 5105, 5107,...",미국
10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990",1990.0,전체 관람가,"[서부, SF, 판타지, 코미디]","[1076, 4603, 1031, 5104, 10001, 5984, 10002, 1...",미국
10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977",1997.0,PG,"[판타지, 모험, SF, 액션]","[1007, 535, 215, 1236, 35]",미국
...,...,...,...,...,...,...,...
10995,공포의 여정,"Journey Into Fear , 1975",NaN,PG,[스릴러],"[2464, 16573, 2101, 10619, 17815, 17814, 16848...",미국
10996,버스틴 루즈,"Bustin' Loose , 1981",NaN,R,[코미디],"[9598, 6520, 506, 11123]",미국
10997,블랙 엔젤,"Mausoleum , 1983",NaN,청소년 관람불가,[공포],"[198255, 17831, 10233, 140473, 31534, 200668, ...",미국


#### (2) 배우별 출연 영화 목록을 만들어라

In [23]:
# korean과 original에 관련한 columns는 불필요함으로 삭제
peoples = peoples.drop('korean', axis=1)
peoples = peoples.drop('original', axis=1)

group_movies = castings.groupby('people')['movie'].agg(list).reset_index()
actor_movies = pd.merge(peoples, group_movies, on='people', how='left')

# NaN값이 있는 행 제거 (= 출연 영화가 없는 배우 제거)
actor_movies = actor_movies.dropna()

actor_movies

,people,movie
0,5,[10062]
2,9,"[10086, 10476, 10639]"
4,15,"[10071, 10072, 10313, 10330, 10561, 10719, 107..."
5,24,"[10336, 10514]"
6,27,[10144]
...,...,...
7165,381096,[10301]
7166,386664,[10776]
7167,390433,[10802]
7168,398299,[10589]


## 문제 2번. 다음의 contents-based recommender system을 구현하라

#### (1) movie 별로 genre의 IDF를 구하라
- genre가 없는 영화는 삭제
- genre의 $IDF = np.log10(전체영화수/각 장르의 영화수)$

In [24]:
# genres 컬럼에서 결측값을 가진 행을 제거
cleaned_movie_table = set_movie_table.dropna(subset=['genres'])
cleaned_movie_table.head()

,title,title_eng,year,grade,genres,people,country
movie,,,,,,,
10001,시네마 천국,"Cinema Paradiso , 1988",2013.0,전체 관람가,"[드라마, 멜로/로맨스]","[4374, 178, 3241, 47952, 47953, 19538, 18991, ...","이탈리아,프랑스"
10002,빽 투 더 퓨쳐,"Back To The Future , 1985",2015.0,12세 관람가,"[SF, 코미디]","[1076, 4603, 917, 8637, 5104, 9986, 7470, 9987]",미국
10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989",2015.0,12세 관람가,"[SF, 코미디]","[1076, 4603, 917, 5104, 391, 5106, 5105, 5107,...",미국
10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990",1990.0,전체 관람가,"[서부, SF, 판타지, 코미디]","[1076, 4603, 1031, 5104, 10001, 5984, 10002, 1...",미국
10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977",1997.0,PG,"[판타지, 모험, SF, 액션]","[1007, 535, 215, 1236, 35]",미국


In [25]:
cleaned_movie_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 964 entries, 10001 to 10999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      964 non-null    object 
 1   title_eng  963 non-null    object 
 2   year       595 non-null    float64
 3   grade      933 non-null    object 
 4   genres     964 non-null    object 
 5   people     962 non-null    object 
 6   country    962 non-null    object 
dtypes: float64(1), object(6)
memory usage: 60.2+ KB


In [26]:
cleaned_movie_table['genres'].isna().sum()

0

- 영화별 장르 DataFrame 구성

In [27]:
genre_representation = pd.DataFrame(cleaned_movie_table['genres'])
genre_representation

,genres
movie,
10001,"[드라마, 멜로/로맨스]"
10002,"[SF, 코미디]"
10003,"[SF, 코미디]"
10004,"[서부, SF, 판타지, 코미디]"
10005,"[판타지, 모험, SF, 액션]"
...,...
10995,[스릴러]
10996,[코미디]
10997,[공포]


- IDF를 구하기 위해 장르별 count

In [28]:
# 개별 장르 count
movie_genre = []
genre_count = dict()

for index, row in genre_representation.iterrows():
    genres = row['genres']
    for genre in genres:
        if genre in genre_count:
            genre_count[genre] += 1
        else :
            genre_count[genre] = 1

genre_count

{'드라마': 529,
 '멜로/로맨스': 172,
 'SF': 80,
 '코미디': 233,
 '서부': 31,
 '판타지': 55,
 '모험': 117,
 '액션': 211,
 '가족': 40,
 '스릴러': 162,
 '범죄': 117,
 '공포': 79,
 '전쟁': 75,
 '미스터리': 48,
 '뮤지컬': 41,
 '느와르': 15,
 '애니메이션': 6,
 '서사': 1,
 '에로': 10,
 '다큐멘터리': 3}

- 장르별 IDF

In [29]:
total_movie = len(genre_representation) # 전체영화수
genre_idf = dict() 

# 장르별 IDF 계산
for genre in genre_count:
    genre_idf[genre] = np.log10(total_movie / genre_count[genre])
    
# genre_count
genre_idf

{'드라마': 0.26062136186764506,
 '멜로/로맨스': 0.7485485869952818,
 'SF': 1.0809870469108873,
 '코미디': 0.6167211128768119,
 '서부': 1.4927153400685582,
 '판타지': 1.243714344408587,
 '모험': 0.915891172156669,
 '액션': 0.6597945786051381,
 '가족': 1.3820170425748683,
 '스릴러': 0.7745620193601999,
 '범죄': 0.915891172156669,
 '공포': 1.0864499426123893,
 '전쟁': 1.1090157705111308,
 '미스터리': 1.3028357965272435,
 '뮤지컬': 1.3712931771830952,
 '느와르': 1.8079857748471495,
 '애니메이션': 2.205925783519187,
 '서사': 2.984077033902831,
 '에로': 1.9840770339028309,
 '다큐멘터리': 2.5069557791831683}

- 영화별 genre IDF 데이터프레임

In [30]:
from tqdm import tqdm

# 초기 데이터프레임 설정
genre_idf_representation = pd.DataFrame(index=genre_representation.index)

# tqdm을 사용하여 진행 상황 표시
for index, row in tqdm(genre_representation.iterrows(), total=genre_representation.shape[0]):
    for genre in row['genres']:
        if genre in genre_idf:
            # 직접 각 컬럼에 값을 설정
            genre_idf_representation.loc[index, genre] = genre_idf[genre]

# 결과 출력
genre_idf_representation

100%|██████████| 964/964 [00:00<00:00, 6736.93it/s]


,드라마,멜로/로맨스,SF,코미디,서부,판타지,모험,액션,가족,스릴러,범죄,공포,전쟁,미스터리,뮤지컬,느와르,애니메이션,서사,에로,다큐멘터리
movie,,,,,,,,,,,,,,,,,,,,
10001,0.260621,0.748549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10002,NaN,NaN,1.080987,0.616721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10003,NaN,NaN,1.080987,0.616721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10004,NaN,NaN,1.080987,0.616721,1.492715,1.243714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10005,NaN,NaN,1.080987,NaN,NaN,1.243714,0.915891,0.659795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.774562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10996,NaN,NaN,NaN,0.616721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.08645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### (2) 장르 기반 영화 cosine 유사도를 구하라.
- from sklearn.metrics.pairwise import cosine_similarity 사용

In [31]:
genre_idf_representation = genre_idf_representation.fillna(0) # NaN값을 0으로 만듬
genre_idf_representation.head()

,드라마,멜로/로맨스,SF,코미디,서부,판타지,모험,액션,가족,스릴러,범죄,공포,전쟁,미스터리,뮤지컬,느와르,애니메이션,서사,에로,다큐멘터리
movie,,,,,,,,,,,,,,,,,,,,
10001,0.260621,0.748549,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10002,0.000000,0.000000,1.080987,0.616721,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10003,0.000000,0.000000,1.080987,0.616721,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10004,0.000000,0.000000,1.080987,0.616721,1.492715,1.243714,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10005,0.000000,0.000000,1.080987,0.000000,0.000000,1.243714,0.915891,0.659795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
from sklearn.metrics.pairwise import cosine_similarity # 사이킷런에서 지원함

def cos_sim_matrix(a, b):
    cos_sim = cosine_similarity(a, b)
    result_df = pd.DataFrame(data=cos_sim, index=[a.index])

    return result_df

In [33]:
print(genre_idf_representation.index)

Index([10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10010,
       ...
       10989, 10991, 10992, 10993, 10994, 10995, 10996, 10997, 10998, 10999],
      dtype='int64', name='movie', length=964)


In [34]:
cs_df = cos_sim_matrix(genre_idf_representation, genre_idf_representation)
cs_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,954,955,956,957,958,959,960,961,962,963
movie,,,,,,,,,,,,,,,,,,,,,
10001,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.789237,0.077066,0.000000,0.000000,0.789237,0.0,0.000000,0.0,0.061493,0.055123
10002,0.0,1.000000,1.000000,0.539378,0.470079,0.470079,0.470079,0.450043,0.470079,0.595351,...,0.304306,0.844390,0.203254,0.000000,0.304306,0.0,0.495542,0.0,0.000000,0.603962
10003,0.0,1.000000,1.000000,0.539378,0.470079,0.470079,0.470079,0.450043,0.470079,0.595351,...,0.304306,0.844390,0.203254,0.000000,0.304306,0.0,0.495542,0.0,0.000000,0.603962
10004,0.0,0.539378,0.539378,1.000000,0.589183,0.589183,0.589183,0.242743,0.589183,0.641814,...,0.164136,0.455446,0.109631,0.646937,0.164136,0.0,0.267284,0.0,0.000000,0.325764
10005,0.0,0.470079,0.470079,0.589183,1.000000,1.000000,1.000000,0.586183,1.000000,0.955491,...,0.000000,0.526126,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.457763,0.376319
10006,0.0,0.470079,0.470079,0.589183,1.000000,1.000000,1.000000,0.586183,1.000000,0.955491,...,0.000000,0.526126,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.457763,0.376319
10007,0.0,0.470079,0.470079,0.589183,1.000000,1.000000,1.000000,0.586183,1.000000,0.955491,...,0.000000,0.526126,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.457763,0.376319
10008,0.0,0.450043,0.450043,0.242743,0.586183,0.586183,0.586183,1.000000,0.586183,0.560092,...,0.000000,0.503701,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.438251,0.360279
10009,0.0,0.470079,0.470079,0.589183,1.000000,1.000000,1.000000,0.586183,1.000000,0.955491,...,0.000000,0.526126,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.457763,0.376319


#### (3) (2)의 결과에서 특정 영화 (예 : 10010)과 유사한 영화 10편을 나열하라. (유사도값 출력)

In [35]:
# 10010 영화 확인
movies.loc[9] # df에서 index / 행에 대한 내용을 추출

movie                      10010
title                      슈퍼맨 3
title_eng    Superman III , 1983
year                         NaN
grade                     전체 관람가
Name: 9, dtype: object

In [36]:
# 10010 영화와 유사한 영화 10편 나열
cs_df[9].sort_values(ascending=False)[:11]

movie
10010    1.000000
10005    0.955491
10006    0.955491
10007    0.955491
10009    0.955491
10653    0.901855
10619    0.855917
10792    0.841675
10096    0.841675
10095    0.841675
10269    0.832591
Name: 9, dtype: float64

### (4) 특정 배우 (예, 리차드 기어 Richard Gere)가 출연한 영화 중 전체 영화 평점이 높은 순의 영화 5편을 추천하라.

In [39]:
# rates 데이터 불러오기
r_cols = ['user', 'movie', 'rate', 'time']
rates = pd.read_csv(rates_file_path, sep=',', names=r_cols, skiprows=1)
rates.head()

,user,movie,rate,time
0,0,10003,7,1494128040
1,0,10004,7,1467529800
2,0,10018,9,1513344120
3,0,10021,9,1424497980
4,0,10022,7,1427627340


In [40]:
# time 제거 
rates = rates.drop('time', axis=1)
rates.head()

,user,movie,rate
0,0,10003,7
1,0,10004,7
2,0,10018,9
3,0,10021,9
4,0,10022,7


In [41]:
# 사용자 수는 2천명으로 제한 (0번 index부터 1999까지)
unique_users = rates['user'].unique()
selected_users = unique_users[:2000]

user_df = rates[rates['user'].isin(selected_users)]
user_df

,user,movie,rate
0,0,10003,7
1,0,10004,7
2,0,10018,9
3,0,10021,9
4,0,10022,7
...,...,...,...
27044,1998,10874,10
27045,1998,10936,10
27046,1999,10001,10
27047,1999,10003,10


In [42]:
user_df.columns.duplicated().sum()  # 중복된 인덱스 수를 확인

0

#### train, test set 분리

In [43]:
# user을 기준으로 일정 비율로 학습, 테스트 데이터 분리
from sklearn.model_selection import train_test_split

X = user_df.copy()
y = user_df['user']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [44]:
# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

#### 모델에 따른 테스트 데이터의 예측 및 실데이터 간의 정확도 계산

In [45]:
# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(X_test['user'], X_test['movie'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs]) # 평점의 배열
    y_true = np.array(X_test['rate']) # 실제로 본 영화의 배열
    return RMSE(y_true, y_pred)

In [46]:
# 중복된 user와 movie 조합 확인
duplicates = X_train[X_train.duplicated(subset=['user', 'movie'])]
print(duplicates)

       user  movie  rate
9952    465  10033     9
10269   465  10936    10
1564     44  10670    10
14692   819  10200     9
9974    465  10048    10
...     ...    ...   ...
13686   739  10335     9
7916    303  10001     1
9449    430  10171    10
23217  1607  10200    10
1318     44  10020     9

[2428 rows x 3 columns]


In [47]:
# 중복된 'user'와 'movie' 조합에 대해 평균을 사용하여 중복 제거 / (중복 때문에 pivot table를 만들수 없는 오류 해결)
X_train = X_train.groupby(['user', 'movie']).agg({'rate': 'mean'}).reset_index()
X_train

,user,movie,rate
0,0,10003,7.0
1,0,10004,7.0
2,0,10018,9.0
3,0,10021,9.0
4,0,10022,7.0
...,...,...,...
19206,1998,10764,9.0
19207,1998,10936,10.0
19208,1999,10001,10.0
19209,1999,10003,10.0


#### 학습데이터 (사용자, 영화, 평점) matrix

In [48]:
# train 데이터로 Full matrix 구하기 
rating_matrix = X_train.pivot(index='user', columns='movie', values='rate')
rating_matrix

movie,10001,10002,10003,10004,10005,10006,10007,10008,10009,10011,...,10971,10975,10978,10979,10980,10981,10983,10985,10994,10998
user,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,7.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN
2,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,10.0,NaN,9.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# 전체 평균으로 예측치를 계산하는 기본 모델 - 영화 평점 평균
def best_seller(user, movie):
    try:
        rate = train_mean[movie]
    except:
        rate = 3.0
    return rate

In [50]:
# 학습데이터에서의 영화별 평점평균
train_mean = X_train.groupby(['movie'])['rate'].mean()
train_mean.head()

movie
10001    9.339660
10002    9.233833
10003    8.942412
10004    8.908421
10005    8.990291
Name: rate, dtype: float64

In [51]:
# 테스트데이터 (사용자 x 영화): 사용자가 평가한 영화(실데이터)와 사용자가 평가한 영화의 학습데이터에서의 평점평균간 오차
score(best_seller)

2.15039517586285

#### 리차드 기어 출연한 영화의 평점 순위를 보기 위한 데이터 병합전 데이터 파악

In [52]:
# 위에서 미리 만들어놓은 merged_people_data 데이터프레임에서 Richard Gere 배우의 번호를 확인
actor_num = merged_people_data.loc[merged_people_data['original'] == 'Richard Gere', 'people'].iloc[0]
print(actor_num) 

1303


In [53]:
# 리차드 기어의 배우번호 : 1303
actor_movies.head()

,people,movie
0,5,[10062]
2,9,"[10086, 10476, 10639]"
4,15,"[10071, 10072, 10313, 10330, 10561, 10719, 107..."
5,24,"[10336, 10514]"
6,27,[10144]


In [54]:
result = actor_movies.loc[actor_movies['people'] == 1303, 'movie']
result

512    [10104, 10132, 10145, 10560, 10643, 10795]
Name: movie, dtype: object

In [55]:
actor_movies.loc[512]

people                                          1303
movie     [10104, 10132, 10145, 10560, 10643, 10795]
Name: 512, dtype: object

In [56]:
X_train.head()

,user,movie,rate
0,0,10003,7.0
1,0,10004,7.0
2,0,10018,9.0
3,0,10021,9.0
4,0,10022,7.0


#### 데이터 병합

In [57]:
actor_movies = actor_movies.explode('movie')
merge_df = pd.merge(actor_movies, X_train, on='movie', how='inner')
merge_df

,people,movie,user,rate
0,5,10062,2,6.0
1,5,10062,19,10.0
2,5,10062,81,6.0
3,5,10062,86,2.0
4,5,10062,92,1.0
...,...,...,...,...
190172,420466,10420,1590,10.0
190173,420466,10420,1635,4.0
190174,420466,10420,1711,3.0
190175,420466,10420,1820,8.0


In [58]:
merge_df.isna().sum()

people    0
movie     0
user      0
rate      0
dtype: int64

#### 리차드 기어가 출연한 영화 중 전체 영화 평점이 높은 순의 영화 5편

In [59]:
# merge_df에서 'people' 값이 1303인 행만 필터링
filtered_df = merge_df[merge_df['people'] == 1303]

# 'movie'와 'people'을 기준으로 'rate'의 평균을 계산
actor = filtered_df.groupby(['movie', 'people'])['rate'].mean()
actor.head()

movie  people
10104  1303      8.107143
10132  1303      8.470721
10145  1303      9.000000
10560  1303      7.000000
10643  1303      6.750000
Name: rate, dtype: float64

In [60]:
# 'rate'로 정렬 (내림차순)
sorted_actor = actor.sort_values(ascending=False)

# 정렬된 결과 출력
print(sorted_actor.head(5))

movie  people
10145  1303      9.000000
10132  1303      8.470721
10104  1303      8.107143
10560  1303      7.000000
10795  1303      7.000000
Name: rate, dtype: float64
